# AgentOps Langchain Agent Implementation

Using AgentOps monitoring with Langchain is simple. We've created a LangchainCallbackHandler that will do all of the heavy lifting!

First we'll import the typical Langchain packages:

In [1]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from dotenv import load_dotenv

/Library/Python/3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


The only difference with using AgentOps is that we'll also import this special Callback Handler

In [2]:
from agentops.langchain_callback_handler import LangchainCallbackHandler

Next, we'll grab our two API keys. You can use dotenv like below or however else you like to load environment variables

In [3]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
AGENTOPS_API_KEY = os.getenv('AGENTOPS_API_KEY')

This is where AgentOps comes into play. Before creating our LLM instance via Langchain, first we'll create an instance of the AO LangchainCallbackHandler.

Pass in your API key, and optionally any tags to describe this session for easier lookup in the AO dashboard. You can also retrieve the `session_id` of the newly created session.

In [4]:
handler = LangchainCallbackHandler(api_key=AGENTOPS_API_KEY, tags=['Langchain Example'])
print("Agent Ops session ID: " + handler.session_id)
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, callbacks=[handler])

Agent Ops session ID: 3c9deaa9-577c-49ac-9783-e7d0b0fecfbd


Finally, let's use our agent! All of the actions will be recorded in the AO Dashboard

In [5]:
agent = initialize_agent([],
                         llm,
                         agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True,
                         handle_parsing_errors=True)

In [6]:
agent.run("Please write me a short poem about secret agents")



> Entering new AgentExecutor chain...
Sure! Here's a short poem about secret agents:

In shadows they dwell, unseen by the eye,
Masters of disguise, their secrets held high.
They weave through the night, like whispers in air,
Agents of mystery, with skills beyond compare.

With steady hands, they handle the unknown,
Unraveling secrets, their cover not blown.
They traverse the globe, in search of the truth,
Stealthily moving, like shadows of youth.

Their mission, you ask? To protect and defend,
The world's safety, until the very end.
With gadgets and wit, they outsmart their foes,
Silent warriors, where nobody knows.

In dark alleys they gather, plans tightly knit,
Their loyalty unwavering, they never quit.
For justice they fight, with honor and grace,
A secret agent's life, a dangerous embrace.

So raise your glass to these guardians unseen,
The heroes of shadows, in suits so pristine.
They sacrifice all, for a world that's unknown,
Secret agents, silent heroes, never alone.

Final 

'Secret agents, guardians of the night,\nA poem to honor their courage and might.'